In [2]:
# load train data 难点
import os
import numpy as np

points=[]
txt_path = '/home/zlh/ws/machine_learning/test_hmm/data/pt_obs_bag5_xyz.txt'
with open(txt_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        one_list = line.split(', ')
        one_point = [float(str_num) for str_num in one_list[0:3]]
        one_point.append(len(one_list)-4)
        for str in one_list[4:-1]:
            one_point.append(int(str))
        one_point.append(int(one_list[-1][0]))
        points.append(one_point)

print('length is {}'.format(len(points)))
for i in range(3):
    print(points[i])
# points_array = np.array((points), dtype=np.float32)
# print(points_array.shape)

length is 1300325
[5.91418, -0.346745, 1.84473, 2, 0, 0]
[6.0511, -0.338421, 1.83835, 2, 0, 0]
[6.13759, -0.334508, 1.83713, 2, 0, 0]


In [6]:
# load down sampling data
import os

down_points_dir = {}
down_txt_path = '/home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5'
file_list = os.listdir(down_txt_path)
for l in file_list:
    file = os.path.join(down_txt_path, l)
    print("process {}".format(file))
    down_points = []
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            one_list = line.split(', ')
            one_point = [float(str_num) for str_num in one_list[0:3]]
            # one_point.append(len(one_list)-4)
            for str in one_list[4:-1]:
                one_point.append(int(str))
            one_point.append(int(one_list[-1][0]))
            down_points.append(one_point)
    down_points_dir[len(one_list)-4] = down_points

print('length is {}'.format(down_points_dir[10][1]) )


process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/33.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/30.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/62.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/77.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/2.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/150.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/277.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/228.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/107.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/23.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/59.txt
process /home/zlh/ws/machine_learning/test_hmm/data/down_sample_set_bag5/173.txt
process /home/zlh/ws/machine_learnin

In [9]:
# unsuperviser
import numpy as np
from hmmlearn import hmm

# initial configure lambda
# Q = {0, 1, 2} 三类
startprob = np.array([0.3, 0.6, 0.1])  # pi 
# n=0 state transport matrix aij
transmat = np.array([[0.8, 0.1, 0.1],
                    [0.1, 0.8, 0.1],
                    [0.1, 0.1, 0.8]])
# n=0 observe prob bj(k)
emissionprob = np.array([[0.76, 0.01, 0.01, 0.1, 0.1, 0.01, 0.01],
                      [0.01, 0.76, 0.01, 0.1, 0.1, 0.01, 0.01],
                      [0.01, 0.01, 0.76, 0.1, 0.1, 0.01, 0.01]])
label_model = hmm.MultinomialHMM(n_components=3, init_params='ste') # 3 classes

label_model.startprob_ = startprob
label_model.transmat_ = transmat
label_model.emissionprob_ = emissionprob
# label_model.fit(down_points_dir[20],len(down_points_dir[20]))
label_model.sample(n_samples=5, )

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


ValueError: Symbol counts should be nonnegative integers

In [ ]:
# train and plot
from hmmlearn import hmm

best_score = best_model = None
n_fits = 50  #多次拟合
np.random.seed(13)
# (discrete) emissions.
for idx in range(n_fits):
    model = hmm.MultinomialHMM(n_components=3, random_state=idx,
        init_params='se')  # don't init transition, set it below
    # we need to initialize with random transition matrix probabilities
    # because the default is an even likelihood transition
    # we know transitions are rare (otherwise the casino would get caught!)
    # so let's have an Dirichlet random prior with an alpha value of
    # (0.1, 0.9) to enforce our assumption transitions happen roughly 10%
    # of the time

    model.fit(X_train) # 投入多样本的train数据集
    score = model.score(X_validate) # 验证
    print(f'Model #{idx}\tScore: {score}')
    if best_score is None or score > best_score: # 多次拟合的结果取最优的
        best_model = model
        best_score = score

print(f'Generated score: {gen_score}\nBest score:      {best_score}')
# fig, ax = plt.subplots()
# ax.plot(gen_states[:500])
# ax.set_title('States over time')
# ax.set_xlabel('Time (# of rolls)')
# ax.set_ylabel('State')
# fig.show()